In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats

<h1> Research Questions </h1>

In [4]:
with open('data/p01-ex-man-export-1608726738974.json', 'r') as outfile:
    data = json.load(outfile)

In [34]:
def create_focusDF(data):
    
    percentage_goals = []
    broken_Focus = []
    distraction_count = []
    nr_of_services = []
    messages_received_in_focus = []
    
    # transform json part to pandas dataframe
    df = pd.json_normalize(data["focusSessions"])
    
    # create new column, which calculates how long a user stayed in focus
    df["minutesInFocus"] = round((df["endTime"] - df["startTime"])/60000,2)
    
    #df["distractionCount"] = 0

    
    for index, row in df.iterrows():
        # calculate percentage of achieved goals
        if len(row['goals']) == 0:
            percentage_goals.append(0)
        else:
            percentage_goals.append(round(len(row['completedGoals'])/len(row['goals']),2))
        
        # calculate broken focus length
        if len(row["brokenFocus"]) == 0:
            broken_Focus.append(0)
        else:
            break_per_focus= 0
            for element in row["brokenFocus"]:
                if len(element) ==2:
                    break_per_focus += ((element[1]-element[0])/60000)
            broken_Focus.append(round(break_per_focus,2))
            
        # get distractionCount
        if len(row["activeWindows"]) == 0:
            distraction_count.append(0)
        else:
            distractionCountPerFocus = 0
            for element in row["activeWindows"]:
                if element["isDistraction"]:
                    distractionCountPerFocus += 1
            distraction_count.append(distractionCountPerFocus)
        
        # get number of used services
        nr_of_services.append(len(row["services"]))
        
        messages_received = 0
        for element in row["services"]:
            # get number of received messages
            messages_received += len(element["messages"])
        messages_received_in_focus.append(messages_received)
            
    
    df["distractionCount"] = distraction_count
    df["numberOfServices"] = nr_of_services
    df["messages_received_in_focus"] = messages_received_in_focus 
    df["brokenFocus"] = broken_Focus     
    df["percentageCompletedGoals"] = percentage_goals
    
    
    
    # drop columns startTime, endTime, originalEndTime, goals and completedGoals
    df = df.drop(["id","services",'startTime', 'endTime','originalEndTime','goals','completedGoals',"activeWindows"], axis=1)
    return df

In [35]:
df = create_focusDF(data)
df

,brokenFocus,rating,comments,scheduled,appVersion,calendarSubject,minutesInFocus,distractionCount,numberOfServices,messages_received_in_focus,percentageCompletedGoals
0,0.00,6,None,False,exman,None,40.00,0,2,1,0.62
1,1.23,4,None,False,exman,None,25.00,0,2,0,0.33
2,0.00,None,None,False,exman,None,55.00,0,2,0,1.00
3,0.00,5,None,False,exman,None,15.00,0,2,1,0.00
4,0.00,5,None,False,exman,None,35.00,0,3,0,0.00
5,0.00,4,None,False,exman,None,25.00,0,3,9,0.00
6,0.00,6,None,False,exman,None,45.00,0,3,0,0.75
7,0.00,5,None,False,exman,None,15.00,0,3,2,1.00
8,0.00,5,None,False,exman,None,25.00,0,3,0,1.00
9,0.00,6,None,False,exman,None,40.00,0,3,6,1.00


In [19]:
len(df["services"][17])

3